## Final Project, Part 1-2:


#### Problem statement

millions of transactions payments to sellers are made everyday on ecommerce platform like shopee or lazada. with the increase in volume, an intelligent system of fraud detection is required for the employees to deal with the mounting transactions volume.

Questions: 
Q1: how to  predict if a tranction is a fraud,using a selected list of given attributed
Q2: is the classification model better than the current Flagging system that flagged transactions that are more than 200,000?

So far 18 out of 18 flagged transactions are indeed Fraud
but 8200 fraud transactions are NOT flagged.

Attribute CO
Target Columns: 'isFraud'

Objective:
Build a Classification Model that can predict if a transaction is fraud or not fraud
    - model 1: Classification model using Linear Regression
    - model 2: Logistic Regression
    - model 3: decision tree?
    
Determine if the Classification model is more accurate compared to the baseline:
    - the baseline is the current flagging system, which detected correctly 18 out of 8218 cases of actual fraud

#### Data Source

https://www.kaggle.com/ntnu-testimon/paysim1

#### Risks or limitations
- Curse of dimentionality:
The dataset might be too thin 6m rows and
- imbalance dataset, the non-fraud observations might be over presented
- unknown variables that are not given in the orginal dataset that might have predicting power over whether a transaction is fraud

### Description of Given Dataset

1. step: Maps a unit of time in the real world. In this case 1 step is 1 hour of time.
2. type: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
3. amount: amount of the transaction in local currency
4. nameOrig: customer who started the transaction
5. oldbalanceOrg: initial balance before the transaction
6. newbalanceOrig: customer's balance after the transaction.
7. nameDest: recipient ID of the transaction. 
8. oldbalanceDest: initial recipient balance before the transaction.
9. newbalanceDest: recipient's balance after the transaction.
10. isFraud: identifies a fraudulent transaction (1) and non fraudulent (0)
11. ifFlaggedFraud: flags illegal attempts to transfer more than 200.000 in a single transaction. 


#### Detailed Steps Taken:
1. 
2. from sklearn.preprocessing import LabelEncoder
3. from sklearn.preprocessing import StandardScaler
4. smote
3. train, test, split
4. .fit

5. sklearn.metrics import classification_report
    - the row with the lower support col is the minority class
    - Recal(sensitity) is the ratio of correctly predicted positive to th all observation 
7. calculate the Recall of the minority class, should definitely zoom in when do the classification report
8. might need to sacrifice some precision for recall, because the cost of not detecting a fraud case is very high

9. can adjust threshold in logistic regression (0.5 to 0.3) to adjust for the cost: if the cost of true positive is higher than false positive (adjust threhold is not frequent)

10. Classification Trees, Decision Forest and use feature_importance_ to determine which features are important in the classifier
Read the classificaiton report, recall of the minority class

> EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path_to_file = 'C:\\Users\\harriet.ly\\Documents\\General Assembly Notes\\GA_Projects\\Projects\\Project 4\\fraud.csv'
dataset = pd.read_csv(path_to_file)
dataset.head(3)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0


In [3]:
# check number of rows and columns
dataset.shape

(6362620, 11)

In [4]:
# Test if there is any null values
dataset.isnull().values.any()

False

In [5]:
#1. Check if there are repeated nameOrig and nameDest, if there are alot, we will use label encode them to use in our model, 
# otherwise, we drop them
dataset['nameOrig'].nunique() # check if there are nameOrig with multiple transactions

6353307

In [6]:
dataset['nameDest'].nunique() # check if there are repeated nameDest

2722362

In [7]:
# There are a lot fewer unique values for nameDest, meaning lots of repeated nameDest, hence we will proceed to encode nameDest 
# to use as an attribute in our model, and drop 'nameOrig' as it is considered as unique as an ID

In [8]:
# Label encoder on nameOrig and nameDest, these two variables are like IDs of the transactions but we will labelencode them to
# include them into the model because they may contain
# important information on the pattern/frequence of the same nameDest or nameOrg. 
# for isntance, highly repeated nameDest is an indicator of fraud

from sklearn import preprocessing

#instantiate
le = preprocessing.LabelEncoder()

#Labelencode, this process might take a while to run
le.fit(dataset['nameDest'])

#nameOrigin_Encoded=le.transform(dataset['nameOrig'])
nameDest_Encoded=le.transform(dataset['nameDest'])

#dataset['nameOrigin_Encoded']=nameOrigin_Encoded
dataset['nameDest_Encoded']=nameDest_Encoded

In [9]:
dataset.head().sort_values(by='nameDest_Encoded',ascending=True)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,nameDest_Encoded
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,391696
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,439685
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,828919
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,1662094
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,1733924


In [10]:
#2. Check if col isFlaggedFraud is completed relatd to Col amount. If amount > 200k, isFlaggedFraud =1
data_flaggedFraud = dataset[dataset['isFlaggedFraud']==1]
data_flaggedFraud[data_flaggedFraud['amount']<200000].shape

(0, 12)

In [11]:
#3. Next, we check if the Col amount is equal to the abs(newbalanceOrig-oldbalanceOrg), In a genuine transaction, the difference between the
# beginning and ending balance of the original account should be the same as the amount of transactions.
# Note that oldbalanceOrg is not necessary always larger than newbalanceOrig, depending on what kind of transactions, hence we use abs()
dataset['error_balanceOrig']= abs(dataset['oldbalanceOrg'] - dataset['newbalanceOrig'])-dataset['amount']
dataset[dataset['error_balanceOrig']>0].shape

(916134, 13)

In [12]:
# there are 1m transactions whose amount do not tally with the origin beginning and ending balances, hence 
# we will include  'error_old_new_balanceOrigin' as an attribute in our fraud detection model

In [13]:
#4. Next, we check if the Col amount is equal to the abs(newbalanceDest-oldbalanceDest)
dataset['error_balanceDest']= abs(dataset['newbalanceDest']-dataset['oldbalanceDest'])-dataset['amount']
dataset[dataset['error_balanceDest']>0].shape

(1567460, 14)

In [14]:
# there are 1m+ transactions whose amount do not tally with the destination beginning and ending balances, hence 
# 'error_old_new_balanceDest' should be used as an attribute in our fraud detection model

In [15]:
#5. Next we check for symmetry if the change in destination account is the same as the change in the original account

dataset['asymmetry']= abs(dataset['newbalanceDest']-dataset['oldbalanceDest']) != abs(dataset['oldbalanceOrg'] - dataset['newbalanceOrig'])

dataset[dataset['isFraud']==1]['asymmetry'].sum()

6036

In [16]:
# there is a very high percentage of fraud case, 6000+ out of 8000 case of fraud has the asymmetry pattern, 
# therefore we will include the asymmetry in our fraud detection model
# although we now need to be extra careful of multicollinearity in our model if correlated attributes are present
# we will look into removing at least one of the 4 attributes: newbalanceDest, oldbalanceDest, oldbalanceOrg, newbalanceOrig

In [17]:
#6. Check on the categorical attribute 'type'
dataset['type'].unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [18]:
#to use get_dummies to transform this categorical col into 5 Cols that take values 0 and 1
dataset=pd.get_dummies(data=dataset, columns=['type'])
dataset.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,nameDest_Encoded,error_balanceOrig,error_balanceDest,asymmetry,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,1662094,1.455192e-11,-9839.64,True,0,0,0,1,0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,1733924,-1.136868e-12,-1864.28,True,0,0,0,1,0
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,439685,0.000000e+00,-181.00,True,0,0,0,0,1
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,391696,0.000000e+00,21001.00,True,0,1,0,0,0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,828919,0.000000e+00,-11668.14,True,0,0,0,1,0


In [19]:
# Lastly in the EDA/preliminary feature selection, we drop attributes columns that are correlated to other attributes
# Note that Assymmetry can actually be determined by error_balanceOrig,error_balanceDest, and amount, therefore we wil drop it too
data = dataset.drop(['nameOrig','nameDest','newbalanceOrig','newbalanceDest','isFlaggedFraud','asymmetry'], axis=1)
data

,step,amount,oldbalanceOrg,oldbalanceDest,isFraud,nameDest_Encoded,error_balanceOrig,error_balanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.00,0.00,0,1662094,1.455192e-11,-9.839640e+03,0,0,0,1,0
1,1,1864.28,21249.00,0.00,0,1733924,-1.136868e-12,-1.864280e+03,0,0,0,1,0
2,1,181.00,181.00,0.00,1,439685,0.000000e+00,-1.810000e+02,0,0,0,0,1
3,1,181.00,181.00,21182.00,1,391696,0.000000e+00,2.100100e+04,0,1,0,0,0
4,1,11668.14,41554.00,0.00,0,828919,0.000000e+00,-1.166814e+04,0,0,0,1,0
5,1,7817.71,53860.00,0.00,0,2247218,-9.094947e-13,-7.817710e+03,0,0,0,1,0
6,1,7107.77,183195.00,0.00,0,2063363,-1.091394e-11,-7.107770e+03,0,0,0,1,0
7,1,7861.64,176087.23,0.00,0,2314008,1.364242e-11,-7.861640e+03,0,0,0,1,0
8,1,4024.36,2671.00,0.00,0,768940,-1.353360e+03,-4.024360e+03,0,0,0,1,0
9,1,5337.77,41720.00,41898.00,0,282960,-3.637979e-12,-3.788560e+03,0,0,1,0,0


> Re-balance using Smote Tomek

In [20]:
Xs= data.drop(['isFraud'], axis=1)

y = data['isFraud']

In [21]:
# out of 6M+ transaction, only 8000+ is fraud, the rest is not fraud. The target col "isFraud" is highly imbalanced towards
# no-fraud transactions, therefore we need to balance the dataset. In this case, we use Smote Tomek to do so

In [ ]:
# Apply SMOTE + Tomek links
from imblearn.combine import SMOTETomek
#instantiate
sm = SMOTETomek()
#fit, this takes a while
X_resampled, y_resampled = sm.fit_sample(Xs, y)

In [ ]:
#normalize/standardize Xs features before applying machine learning techniques
#scaling is especially
#from sklearn.preprocessing import StandardScaler
#import numpy as np

#scaler = StandardScaler() # instantiate
#scaled_Xs = scaler.fit_transform(Xs) #fit

#scaled_Xs = pd.DataFrame(scaled_Xs, columns=feature_cols)

In [ ]:
# decision tree
#clf = XGBClassifier(max_depth = 3, scale_pos_weight = weights, \
                n_jobs = 4)